In [20]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Definición de la ecuación
def An(T, A, B, C, D, E, F, G):
    return A + (B / (T + C)) + (D*T) + (E * np.log(T)) + (F * (T**G))

especie1 = '2-PE'
especie2 = '1,2-PP'

# Datos de (presión en kPa y temperatura en K) tabla 3 del articulo
datos = {
    'T1': np.array([362.80, 369.85, 376.59, 380.32, 384.40, 388.37, 391.21, 394.47, 396.92, 399.26, 402.22, 405.08, 408.97, 414.71, 422.51, 432.38, 445.60, 459.29, 469.69, 480.96, 488.05, 491.95]),
    'P1': np.array([0.77, 0.97, 1.61, 1.91, 2.21, 2.67, 3.14, 3.67, 4.21, 4.57, 5.17, 5.77, 6.89, 8.77, 11.77, 16.81, 26.67, 41.44, 56.47, 76.64, 92.10, 101.33]),
    'T2': np.array([358.15, 364.55, 371.85, 375.88, 380.04, 384.10, 390.50, 393.66, 395.19,398.14, 401.10, 404.98, 411.02, 419.01,428.99, 442.19, 456.07, 466.88, 469.69, 478.35, 485.75, 489.80]),
    'P2': np.array([0.92, 1.25, 1.73, 2.13, 2.50, 2.97, 3.43, 3.97, 4.63, 4.90, 5.42, 6.10, 7.27, 8.98, 11.87, 16.67, 26.07, 39.64, 54.60, 75.07, 91.06, 101.33]),
}

df = pd.DataFrame(datos)

# Estimación inicial de los coeficientes usando los de la tabla 2 del articulo
coeficientes_iniciales = [  58.17, -8985.10, 0, 0,-5.7034,  4.6087e-18, 6.0]
coeficientes_iniciales2 = [ 133.75, -11577.65, 0, 0, -17.5651,  1.3782e-5, 2.0]
# Ajuste de la curva a los datos para la Especie1
parametros_optimos_1, covarianza_1 = curve_fit(An, df['T1'], np.log(df['P1']), p0=coeficientes_iniciales, maxfev=10000)
A_1, B_1, C_1, D_1, E_1, F_1, G_1 = parametros_optimos_1

# Ajuste de la curva a los datos para la Especie2
parametros_optimos_2, covarianza_2 = curve_fit(An, df['T2'], np.log(df['P2']), p0=coeficientes_iniciales2, maxfev=20000)
A_2, B_2, C_2, D_2, E_2, F_2, G_2 = parametros_optimos_2

# Crear DataFrame con los resultados
resultados = pd.DataFrame({
    'Especie': [especie1, especie2],
    'A': [A_1, A_2],
    'B': [B_1, B_2],
    'C': [C_1, C_2],
    'D': [D_1, D_2],
    'E': [E_1, E_2],
    'F': [F_1, F_2],
    'G': [G_1, G_2]
})

display(resultados)


,Especie,A,B,C,D,E,F,G
0,2-PE,172.178337,-21889.580987,74.140360,-0.001585,-20.668471,-2.106893e-16,3.306347
1,"1,2-PP",798.878069,-11553.927690,-151.213816,-0.011527,-130.675626,2.391000e-04,1.993215


In [65]:
import numpy as np
import pandas as pd

# Definir función para calcular la presión de saturación con la ecuación de Antoine:
def Psat(A, B, C, D, E, F, G, T):
    return np.exp(A + (B / (T + C)) + (D*T) + (E * np.log(T)) + (F * (T**G)))

# Datos experimentales:
P = 101.325  # Pa

# Parámetros de Antoine para cada componente en cada mezcla
An1 = np.array([58.17, -8985.10, 0, 0, -5.7034, 4.6087e-18, 6])
An2 = np.array([154.28, -13145.00, 0, 0, -20.1810, 1.0283e-5, 2])
An3 = np.array([133.44, -13446.00, 0, 0, -16.6590, 6.6391e-6, 2])
An4 = np.array([133.75, -11577.65, 0, 0, -17.5651, 1.3782e-6, 2])

# Datos para las cuatro mezclas:
mezclas = [
    {'nombre': '2-PE (1) + 2-EP (2)',
     'T_exp': np.array([477.80, 477.80, 479.27, 479.77, 480.08, 481.66,483.18, 484.93, 485.63, 487.09, 487.85, 488.22, 489.45, 490.27, 490.95, 491.50, 491.65,491.70 ,491.78 , 491.95]),
     'x': np.array([0,  0.0269,0.0650, 0.0865,  0.1004, 0.1688, 0.2407, 0.3151,  0.3573,  0.4488, 0.5054, 0.5317, 0.6381,  0.7156, 0.7969, 0.8958,  0.9303,0.9503, 0.9679, 1.0000]),
     'y': np.array([0, 0.0115,0.0316, 0.0437,  0.0531,  0.0988, 0.1609, 0.2292,0.2754, 0.3676, 0.4285, 0.4593, 0.5784, 0.6718, 0.7692, 0.8842, 0.9228, 0.9452 ,  0.9648, 1]),
     'Antoine_params': [An1, An2]},
    #done
     {'nombre': '1,2-PP (1) + 2-EP (2)',
     'T_exp': np.array([477.80, 477.80, 479.27, 479.77, 480.08, 481.66,483.18, 484.93, 485.63, 487.09, 487.85, 488.22, 489.45, 490.27, 490.95, 491.50, 491.65,491.70 ,491.78 , 491.95]),
      'x': np.array([0,  0.0269,0.0650, 0.0865,  0.1004, 0.1688, 0.2407, 0.3151,  0.3573,  0.4488, 0.5054, 0.5317, 0.6381,  0.7156, 0.7969, 0.8958,  0.9303,0.9503, 0.9679, 1.0000]),
     'y': np.array([0, 0.0115,0.0316, 0.0437,  0.0531,  0.0988, 0.1609, 0.2292,0.2754, 0.3676, 0.4285, 0.4593, 0.5784, 0.6718, 0.7692, 0.8842, 0.9228, 0.9452 ,  0.9648, 1]),
     'Antoine_params': [An4, An2]},
     {'nombre': '2-PE (1) + 1,2-PP (2)',
     'T_exp': np.array([477.80, 477.80, 479.27, 479.77, 480.08, 481.66,483.18, 484.93, 485.63, 487.09, 487.85, 488.22, 489.45, 490.27, 490.95, 491.50, 491.65,491.70 ,491.78 , 491.95]),
     'x': np.array([0,  0.0269,0.0650, 0.0865,  0.1004, 0.1688, 0.2407, 0.3151,  0.3573,  0.4488, 0.5054, 0.5317, 0.6381,  0.7156, 0.7969, 0.8958,  0.9303,0.9503, 0.9679, 1.0000]),
     'y': np.array([0, 0.0115,0.0316, 0.0437,  0.0531,  0.0988, 0.1609, 0.2292,0.2754, 0.3676, 0.4285, 0.4593, 0.5784, 0.6718, 0.7692, 0.8842, 0.9228, 0.9452 ,  0.9648, 1]),
     'Antoine_params': [An1, An4]},
     {'nombre': '2,1-PP (1) + 2-EP (2)',
     'T_exp': np.array([477.80, 477.80, 479.27, 479.77, 480.08, 481.66,483.18, 484.93, 485.63, 487.09, 487.85, 488.22, 489.45, 490.27, 490.95, 491.50, 491.65,491.70 ,491.78 , 491.95]),
      'x': np.array([0,  0.0269,0.0650, 0.0865,  0.1004, 0.1688, 0.2407, 0.3151,  0.3573,  0.4488, 0.5054, 0.5317, 0.6381,  0.7156, 0.7969, 0.8958,  0.9303,0.9503, 0.9679, 1.0000]),
     'y': np.array([0, 0.0115,0.0316, 0.0437,  0.0531,  0.0988, 0.1609, 0.2292,0.2754, 0.3676, 0.4285, 0.4593, 0.5784, 0.6718, 0.7692, 0.8842, 0.9228, 0.9452 ,  0.9648, 1]),
     'Antoine_params': [An3, An2]},
     
    # Agregar datos y parámetros para las otras tres mezclas
]

# Almacenar resultados en un DataFrame
dataframes_mezclas = {}

for mezcla in mezclas:
    nombre = mezcla['nombre']
    T_exp = mezcla['T_exp']
    x = mezcla['x']
    y = mezcla['y']
    Antoine_params = mezcla['Antoine_params']
    resultados = []

    for j in range(len(T_exp)):
        fila = {'T_exp': T_exp[j], 'x': x[j], 'y': y[j]}

        # Calcular Psat para cada componente de la mezcla
        Psat_i = [Psat(*Ant_params, T=T_exp[j]) for Ant_params in Antoine_params]
        # Calcular g1 para cada componente de la mezcla
        for k, (xi, yi, Psat_k) in enumerate(zip(x[j:], y[j:], Psat_i), start=1):
            # Manejar división por cero
            g1 = np.nan if xi == 0 or Psat_k == 0 else yi * P / xi / Psat_k
            fila[f'g{k}'] = g1

        # Calcular g2 con la ecuación modificada
        if (1 - x[j]) == 0 or any(Psat_k == 0 for Psat_k in Psat_i):
            g2 = np.nan  # Si hay división por cero, asignar NaN
        else:
            g2 = (1 - y[j]) * P / (1 - x[j]) / Psat_k
        fila['g2'] = g2

        resultados.append(fila)

    # Crear DataFrame para cada mezcla
    df_mezcla = pd.DataFrame(resultados)
    dataframes_mezclas[nombre] = df_mezcla

# Acceder a los DataFrames individuales por nombre de mezcla
print("DataFrames por mezcla:")
for nombre, df_mezcla in dataframes_mezclas.items():
    display(f"\n{nombre}:", df_mezcla)


DataFrames por mezcla:


'\n2-PE (1) + 2-EP (2):'

,T_exp,x,y,g1,g2
0,477.80,0.0000,0.0000,NaN,1.002781
1,477.80,0.0269,0.0115,0.627657,1.018651
2,479.27,0.0650,0.0316,0.684963,1.001021
3,479.77,0.0865,0.0437,0.701946,0.999239
4,480.08,0.1004,0.0531,0.728539,0.996982
5,481.66,0.1688,0.0988,0.771728,0.987535
6,483.18,0.2407,0.1609,0.845280,0.969655
7,484.93,0.3151,0.2292,0.876878,0.946267
8,485.63,0.3573,0.2754,0.911699,0.932027
9,487.09,0.4488,0.3676,0.931358,0.915700


'\n1,2-PP (1) + 2-EP (2):'

,T_exp,x,y,g1,g2
0,477.80,0.0000,0.0000,NaN,1.002781
1,477.80,0.0269,0.0115,9.949904,1.018651
2,479.27,0.0650,0.0316,11.065254,1.001021
3,479.77,0.0865,0.0437,11.412654,0.999239
4,480.08,0.1004,0.0531,11.892253,0.996982
5,481.66,0.1688,0.0988,12.855434,0.987535
6,483.18,0.2407,0.1609,14.358208,0.969655
7,484.93,0.3151,0.2292,15.233426,0.946267
8,485.63,0.3573,0.2754,15.981356,0.932027
9,487.09,0.4488,0.3676,16.634919,0.915700


'\n2-PE (1) + 1,2-PP (2):'

,T_exp,x,y,g1,g2
0,477.80,0.0000,0.0000,NaN,23.274124
1,477.80,0.0269,0.0115,0.627657,23.642453
2,479.27,0.0650,0.0316,0.684963,23.573867
3,479.77,0.0865,0.0437,0.701946,23.648681
4,480.08,0.1004,0.0531,0.728539,23.667805
5,481.66,0.1688,0.0988,0.771728,23.813206
6,483.18,0.2407,0.1609,0.845280,23.736714
7,484.93,0.3151,0.2292,0.876878,23.569258
8,485.63,0.3573,0.2754,0.911699,23.376137
9,487.09,0.4488,0.3676,0.931358,23.301329


'\n2,1-PP (1) + 2-EP (2):'

,T_exp,x,y,g1,g2
0,477.80,0.0000,0.0000,NaN,1.002781
1,477.80,0.0269,0.0115,0.761055,1.018651
2,479.27,0.0650,0.0316,0.827802,1.001021
3,479.77,0.0865,0.0437,0.847386,0.999239
4,480.08,0.1004,0.0531,0.878886,0.996982
5,481.66,0.1688,0.0988,0.927783,0.987535
6,483.18,0.2407,0.1609,1.012907,0.969655
7,484.93,0.3151,0.2292,1.046919,0.946267
8,485.63,0.3573,0.2754,1.086920,0.932027
9,487.09,0.4488,0.3676,1.107058,0.915700
